# Introduction to Julia

#### John Pearson
#### StatML Group Meeting
#### October 8, 2015

# What is Julia?

- new programming language (public since 2012)
- brought to you by MIT, Julia Computing
- focused on scientific computing applications
- aims to be a high-level prototyping language that is also fast enough for serious simulation
    - as easy to learn and flexible as Python or R
    - within 2x the speed of C, C++, or FORTRAN
- Currently (almost) 0.4, with very active developer community (more later)


# Reasons to Consider Julia

- you mostly write your own algorithms
- you need to write code that is very fast
- you’d like to prototype and optimize within the same language
- you want zero boilerplate calls to C, Python (and soon C++)
- you need powerful metaprogramming/macros
- you want access to state-of-the-art automatic differentiation and optimization
- you want to use Greek letters in variable names
- you are interested in programming language design
- you want to get in on the ground floor


# Reasons Julia may not be for you (yet)

- you mostly use packages written by others
- you rely critically on several packages that aren't available
- you don’t have much need to optimize your code
- you perform mostly data analysis and model building, as opposed to algorithm development
- you prioritize stability and reliability 
    - (or don’t write a lot of tests for your own code)
- you don’t want to devote time to keeping up with changes
- you have zero background with compiled languages

# What makes Julia different?

- just in time (JIT) compilation
- rich type system
- multiple dispatch
- metaprogramming (programs that write programs: think C macros, STAN)
- no more fear of writing loops

More on each of these later!

# Learning Julia

- [Julia homepage](http://julialang.org/)
- [Learning resources](http://julialang.org/learning/)
- [Docs](http://docs.julialang.org/en/release-0.3/) 
    - about to change to 0.4 
    - more comprehensive and up-to-date than most other info
- [YouTube](https://www.youtube.com/user/JuliaLanguage)
- [GitHub](https://github.com/JuliaLang/julia)
    - this is where most of the development discussion takes place
    - being GitHub-friendly currently important
- [julia-users list](https://groups.google.com/forum/#!forum/julia-users)